<a href="https://colab.research.google.com/github/astrapi69/DroidBallet/blob/master/NLP_D1_2_LC10_Applications_Text_Classification_ML_and_Basic_NNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><a target="_blank" href="https://learning.constructor.org/"><img src="https://drive.google.com/uc?id=1RNy-ds7KWXFs7YheGo9OQwO3OnpvRSU1" width="200" style="background:none; border:none; box-shadow:none;" /></a> </center>

_____

<center>Constructor Academy, 2024</center>

# Sentiment Analysis - Machine Learning and Basic Deep Neural Network Models

We have already discussed that sentiment analysis, also popularly known as opinion analysis or opinion mining is one of the most important applications of NLP. The key idea is to predict the potential sentiment of a body of text based on the textual content. In this sub-unit, we will be exploring supervised learning models.

![](https://github.com/dipanjanS/nlp_workshop_dhs18/blob/master/Unit%2012%20-%20Project%209%20-%20Sentiment%20Analysis%20-%20Supervised%20Learning/sentiment_cover.png?raw=1)

Another way to build a model to understand the text content and predict the sentiment of the text based reviews is to use supervised machine learning. To be more specific, we will be using classification models for solving this problem. We will be building an automated sentiment text classification system in subsequent sections. The major steps to achieve this are mentioned as follows.

1.	Prepare train and test datasets (optionally a validation dataset)
2.	Pre-process and normalize text documents
3.	Feature Engineering
4.	Model training
5.	Model prediction and evaluation

These are the major steps for building our system. Optionally the last step would be to deploy the model in your server or on the cloud. The following figure shows a detailed workflow for building a standard text classification system with supervised learning (classification) models.

![](https://github.com/dipanjanS/nlp_workshop_dhs18/blob/master/Unit%2012%20-%20Project%209%20-%20Sentiment%20Analysis%20-%20Supervised%20Learning/sentiment_classifier_workflow.png?raw=1)


In our scenario, documents indicate the movie reviews and classes indicate the review sentiments which can either be positive or negative making it a binary classification problem. We will build models using both traditional machine learning methods and newer deep learning in the subsequent sections.

In [ ]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 20.0 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Load and View Dataset

In [ ]:
import pandas as pd

dataset = pd.read_csv(r'https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Build Train and Test Datasets

In [ ]:
# build train and test datasets
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

# you can also use train test split here to split the data
# train the model on 35000 reviews - sentiment labels
# test the performance of the trained model on 15000 reviews
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]

test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

## Text Wrangling & Normalization

In [ ]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    # remove HTML tags
    doc = strip_html_tags(doc)
    # remove extra newlines and tabs \ spaces
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    # lowercase the text
    doc = doc.lower()
    # remove accented characters from text => convert to plain english
    doc = remove_accented_chars(doc)
    # expand contractions i.e. won't => would not
    doc = contractions.fix(doc)

    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, flags=re.I|re.A)
    doc = re.sub(' +', ' ', doc) # remove extra spaces between words
    doc = doc.strip()

    norm_docs.append(doc)

  return norm_docs

In [ ]:
%%time

norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

  2%|▏         | 869/35000 [00:00<00:15, 2217.58it/s]<ipython-input-5-3be18ca5d74d>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
100%|██████████| 15000/15000 [00:09<00:00, 1579.16it/s]

CPU times: user 30.2 s, sys: 236 ms, total: 30.5 s
Wall time: 38.8 s


## Traditional Supervised Machine Learning Models

### Feature Engineering

In [ ]:
%%time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
# min_df=5, max_df=1.0 -> remove words occuring in < 5 documents (very rare words), keep words even if occuring in 100% (all) of docs
# ngram_range=(1,2) -> words and bigrams as features
cv_train_features = cv.fit_transform(norm_train_reviews)



# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

CPU times: user 29.6 s, sys: 1.12 s, total: 30.7 s
Wall time: 30.8 s


In [ ]:
%%time

# transform test reviews into features
# we use .transform(..) and NOT .fit_transform(..)
# to use vocabulary learnt during training data as our features
cv_test_features = cv.transform(norm_test_reviews)


tv_test_features = tv.transform(norm_test_reviews)

CPU times: user 11 s, sys: 112 ms, total: 11.1 s
Wall time: 11.1 s


In [ ]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)


print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (35000, 194960)  Test features shape: (15000, 194960)
TFIDF model:> Train features shape: (35000, 194960)  Test features shape: (15000, 194960)


### Model Training, Prediction and Performance Evaluation

### Try out Logistic Regression

The logistic regression model is actually a statistical model developed by statistician
David Cox in 1958. It is also known as the logit or logistic model since it uses the
logistic (popularly also known as sigmoid) mathematical function to estimate the
parameter values. These are the coefficients of all our features such that the overall loss
is minimized when predicting the outcome—

In [ ]:
%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1,
                        solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features, train_sentiments)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features)

CPU times: user 1min 11s, sys: 1min 11s, total: 2min 23s
Wall time: 1min 20s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
lr_bow_predictions[:10], test_sentiments[:10]

(array(['negative', 'positive', 'negative', 'positive', 'positive',
        'negative', 'positive', 'positive', 'positive', 'positive'],
       dtype=object),
 array(['negative', 'positive', 'negative', 'positive', 'positive',
        'negative', 'positive', 'positive', 'positive', 'negative'],
       dtype=object))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.90      0.90      0.90      7490
    positive       0.90      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



,negative,positive
negative,6761,729
positive,713,6797


### Try some feature selection and see effect on performance

In [ ]:
# build BOW features on train reviews
cv2 = CountVectorizer(binary=False, min_df=0.01, max_df=0.9, ngram_range=(1,2))
# min_df=0.01 => remove words occuring in < 1% of documents,
# max_df=0.9 => remove words occuring in > 90% of docs
# words as features
cv_train_features2 = cv2.fit_transform(norm_train_reviews)
# transform test reviews into features
cv_test_features2 = cv2.transform(norm_test_reviews)
print('BOW model:> Train features shape:', cv_train_features2.shape, ' Test features shape:', cv_test_features2.shape)

BOW model:> Train features shape: (35000, 3350)  Test features shape: (15000, 3350)


In [ ]:
3350 / 194960 # features reduced to 1.7% from original featureset of 194K features,

0.017183011899876897

In [ ]:
%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1,
                        solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features2, train_sentiments)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features2)

CPU times: user 8.51 s, sys: 25.6 ms, total: 8.54 s
Wall time: 8.86 s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.87      0.87      0.87      7490
    positive       0.87      0.87      0.87      7510

    accuracy                           0.87     15000
   macro avg       0.87      0.87      0.87     15000
weighted avg       0.87      0.87      0.87     15000



,negative,positive
negative,6519,971
positive,961,6549


In [ ]:
%%time

# Logistic Regression model on TF-IDF features

# train model
lr.fit(tv_train_features, train_sentiments)

# predict on test data
lr_tfidf_predictions = lr.predict(tv_test_features)

CPU times: user 5.02 s, sys: 4.3 s, total: 9.33 s
Wall time: 5.46 s


In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.91      0.89      0.90      7490
    positive       0.90      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



,negative,positive
negative,6695,795
positive,674,6836


### Try out Random Forest

Decision trees are a family of supervised machine learning algorithms that can represent
and interpret sets of rules automatically from the underlying data. They use metrics like
information gain and gini-index to build the tree. However, a major drawback of decision
trees is that since they are non-parametric, the more data there is, greater the depth of
the tree. We can end up with really huge and deep trees that are prone to overfitting. The
model might work really well on training data, but instead of learning, it just memorizes
all the training samples and builds very specific rules to them. Hence, it performs really
poorly on the test data. Random forests try to tackle this problem.

A random forest is a meta-estimator or an ensemble model that fits a number of
decision tree classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting. The sub-sample size is always
the same as the original input sample size, but the samples are drawn with replacement
(bootstrap samples). In random forests, all the trees are trained in parallel (bagging
model/bootstrap aggregation). Besides this, each tree in the ensemble is built from a
sample drawn with replacement (i.e., a bootstrap sample) from the training set. Also,
when splitting a node during the construction of the tree, the split that is chosen is no
longer the best split among all features. Instead, the split that is picked is the best split
among a random subset of the features. T

In [ ]:
%%time

# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier

# instantiate model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# train model
rf.fit(cv_train_features, train_sentiments)

# predict on test data
rf_bow_predictions = rf.predict(cv_test_features)

CPU times: user 3min 28s, sys: 421 ms, total: 3min 28s
Wall time: 1min 59s


In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.86      0.86      0.86      7490
    positive       0.86      0.86      0.86      7510

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



,negative,positive
negative,6431,1059
positive,1086,6424


In [ ]:
%%time

# Random Forest model on TF-IDF features

# train model
rf.fit(tv_train_features, train_sentiments)

# predict on test data
rf_tfidf_predictions = rf.predict(tv_test_features)

CPU times: user 3min 8s, sys: 347 ms, total: 3min 9s
Wall time: 1min 45s


In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.84      0.86      0.85      7490
    positive       0.86      0.84      0.85      7510

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000



,negative,positive
negative,6476,1014
positive,1199,6311


## Newer Supervised Deep Learning Models

In [ ]:
import gensim
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder

## Prediction class label encoding

In [ ]:
le = LabelEncoder()
# tokenize train reviews & encode train labels
tokenized_train = [nltk.word_tokenize(text)
                       for text in norm_train_reviews]
y_train = le.fit_transform(train_sentiments)
# tokenize test reviews & encode test labels
tokenized_test = [nltk.word_tokenize(text)
                       for text in norm_test_reviews]
y_test = le.fit_transform(test_sentiments)

In [ ]:
# print class label encoding map and encoded labels
print('Sentiment class label map:', dict(zip(le.classes_, le.transform(le.classes_))))
print('Sample test label transformation:\n'+'-'*35,
      '\nActual Labels:', test_sentiments[:3], '\nEncoded Labels:', y_test[:3])

Sentiment class label map: {'negative': 0, 'positive': 1}
Sample test label transformation:
----------------------------------- 
Actual Labels: ['negative' 'positive' 'negative'] 
Encoded Labels: [0 1 0]


## Feature Engineering with word embeddings

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
%%time
# build word2vec model - takes close to 5-6 mins
w2v_num_features = 300
w2v_model = gensim.models.Word2Vec(tokenized_train, vector_size=w2v_num_features, window=150,
                                   min_count=10, workers=4, epochs=5)

CPU times: user 9min 15s, sys: 890 ms, total: 9min 16s
Wall time: 5min 17s


In [ ]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    # get the vocabulary of all unique words in your training data
    vocabulary = set(model.wv.index_to_key)

    # function to generate averaged word embeddings for every document (a row in your X)
    def average_word_vectors(words, model, vocabulary, num_features):
        # averaged document embedding vector
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0. # total number of words in a document

        # for each word in a document
        for word in words:
            # if the word exists in your vocabulary
            if word in vocabulary:
                # total words increases by 1
                nwords = nwords + 1.
                # add \ sum up word embedding for the matched word
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            # generate averaged document embedding => sum embedding / num of words
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [ ]:
# generate averaged word vector features from word2vec model
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)

In [ ]:
print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape, ' Test features shape:', avg_wv_test_features.shape)

Word2Vec model:> Train features shape: (35000, 300)  Test features shape: (15000, 300)


In [ ]:
300 /  194960 # features reduced to 0.15% from original featureset of 194K features,

0.0015387771850636028

## Modeling with deep neural networks

### Building Deep neural network architecture

In [ ]:
#[300] - [512]-[256]-[256] - [1] => 0/1

In [ ]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, input_shape=(num_input_features,)))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))

    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))

    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))

    dnn_model.add(Dense(1))
    dnn_model.add(Activation('sigmoid'))

    dnn_model.compile(loss='binary_crossentropy', optimizer='adam',
                      metrics=['accuracy'])
    return dnn_model

In [ ]:
w2v_dnn = construct_deepnn_architecture(num_input_features=w2v_num_features)

### Visualize sample deep architecture

In [ ]:
w2v_dnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               154112    
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               6

### Model Training, Prediction and Performance Evaluation

In [ ]:
batch_size = 100
w2v_dnn.fit(avg_wv_train_features, y_train, epochs=10, batch_size=batch_size,
            shuffle=True, validation_split=0.1, verbose=1)

Epoch 1/10
315/315 [==============================] - 7s 4ms/step - loss: 0.3282 - accuracy: 0.8606 - val_loss: 0.2959 - val_accuracy: 0.8746
Epoch 2/10
315/315 [==============================] - 1s 3ms/step - loss: 0.2948 - accuracy: 0.8765 - val_loss: 0.2912 - val_accuracy: 0.8817
Epoch 3/10
315/315 [==============================] - 1s 3ms/step - loss: 0.2857 - accuracy: 0.8820 - val_loss: 0.2903 - val_accuracy: 0.8843
Epoch 4/10
315/315 [==============================] - 1s 4ms/step - loss: 0.2769 - accuracy: 0.8831 - val_loss: 0.2910 - val_accuracy: 0.8786
Epoch 5/10
315/315 [==============================] - 1s 5ms/step - loss: 0.2704 - accuracy: 0.8879 - val_loss: 0.2884 - val_accuracy: 0.8840
Epoch 6/10
315/315 [==============================] - 1s 3ms/step - loss: 0.2638 - accuracy: 0.8917 - val_loss: 0.3031 - val_accuracy: 0.8734
Epoch 7/10
315/315 [==============================] - 1s 4ms/step - loss: 0.2572 - accuracy: 0.8945 - val_loss: 0.3160 - val_accuracy: 0.8797
Epoch 

In [ ]:
y_pred = w2v_dnn.predict(avg_wv_test_features)
y_pred = y_pred.ravel()
y_pred[:10]

469/469 [==============================] - 1s 2ms/step


array([0.03068882, 0.9998275 , 0.05138788, 0.99414223, 0.9823267 ,
       0.02264179, 0.9999064 , 0.67065805, 0.95178527, 0.842616  ],
      dtype=float32)

In [ ]:
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred]
y_pred[:10]

[0, 1, 0, 1, 1, 0, 1, 1, 1, 1]

In [ ]:
predictions = le.inverse_transform(y_pred)

In [ ]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.89      0.86      0.88      7490
    positive       0.87      0.90      0.88      7510

    accuracy                           0.88     15000
   macro avg       0.88      0.88      0.88     15000
weighted avg       0.88      0.88      0.88     15000



,negative,positive
negative,6471,1019
positive,767,6743
